In [8]:
import requests
import re
import time
import random
import json
import os
cong_key='Jq9Uy2kgbNouvTJfoixzEYYQQJ88FAjM3D5HFPne'

In [9]:
cong={}
next='https://api.congress.gov/v3/member?'
while True:
    member_resp=requests.get(f'{next}&api_key={cong_key}')
    for m in member_resp.json()['members']:
        cong[re.sub(r'[^a-zA-Z ]','',m['name'].lower()).strip()]=m['bioguideId']
    next=member_resp.json().get('pagination').get('next')
    if not next:
        break
    time.sleep(random.random()*2)

In [10]:
with open('./opensecrets_info.json', 'r') as f:
    cong_info=json.load(f)

In [11]:
from string import ascii_lowercase as alc
mapping={}
manual=[]
for o_id,info in cong_info.items():
    name=re.sub(r'[^a-zA-Z ]','',info['name'].lower()).strip()
    f,*n=name.split()
    if len(n)==1:
        new=" ".join([n[-1],f])
    elif len(n)==2:
        new=" ".join([n[-1],f,n[0]])

    #guess cycle
    if not cong.get(new):
        new+='  '
        for let in alc:
            new=new[:-1]+let
            if cong.get(new):break
    mapping[o_id]={'id':cong.get(new),'name':info['name']}
    if not cong.get(new):
        manual.append(info['name'])

In [12]:
usable={k:v for k,v in mapping.items() if v['id']}
new_info={v['id']:cong_info[k]['industry'] for k,v in usable.items()}

In [13]:
bill_id=set()
bills=[]
for file in os.listdir('./votes'):
    with open(f'./votes/{file}') as f:
        for z in json.load(f):
            if z[0] not in bill_id:
                bill_id.add(z[0])
                bills.append(z)

In [14]:
new_info={k:v for k,v in new_info.items() if v}
with open('./gov_id_industry.json', 'w') as f:
    json.dump(new_info,f)

In [18]:
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import PorterStemmer

stem=PorterStemmer()
stopwords = set(stopwords.words('english'))
cleaner = lambda sentence: set(re.sub(r'[^a-zA-Z ]',' ',sentence.lower()).split()) - stopwords
term_to_industry={}

for id, info in new_info.items():
    for i in info:
        for term in cleaner(i['industry_name']):
            term=stem.stem(term)
            if not term_to_industry.get(term):
                term_to_industry[term]=set([i['industry_name']])
            else:
                term_to_industry[term].add(i['industry_name'])

In [19]:
term_to_industry

{'retir': {'Retired'},
 'real': {'Real Estate'},
 'estat': {'Real Estate'},
 'pac': {'Leadership PACs'},
 'leadership': {'Leadership PACs'},
 'gambl': {'Casinos/Gambling'},
 'casino': {'Casinos/Gambling'},
 'defens': {'Defense Aerospace', 'Defense Electronics', 'Misc Defense'},
 'misc': {'Misc Defense',
  'Misc Finance',
  'Misc Issues',
  'Misc Manufacturing &amp; Distributing',
  'Misc Transport',
  'Misc Unions'},
 'product': {'Agricultural Services/Products',
  'Crop Production &amp; Basic Processing',
  'Forestry &amp; Forest Products',
  'Pharmaceuticals/Health Products'},
 'basic': {'Crop Production &amp; Basic Processing'},
 'amp': {'Beer, Wine &amp; Liquor',
  'Building Materials &amp; Equipment',
  'Crop Production &amp; Basic Processing',
  'Electronics Mfg &amp; Equip',
  'Fisheries &amp; Wildlife',
  'Food &amp; Beverage',
  'Food Processing &amp; Sales',
  'Forestry &amp; Forest Products',
  'Misc Manufacturing &amp; Distributing',
  'Oil &amp; Gas',
  'Poultry &amp; Eggs

In [20]:
#score, total tuple for each industry
lobbying_scores={id:{i['industry_name']:[0,0] for i in info} for id, info in new_info.items()}
#vader
sid = SentimentIntensityAnalyzer()
neg_votes=set(['Nay','No'])
pos_votes=set(['Aye','Yea'])

for bill in bills:
    #determine sentiment
    sentence=tokenize.sent_tokenize(bill[3])[0]
    ss = sid.polarity_scores(sentence)
    #1 for positive, 0 for negative
    max_sentiment=-1
    if ss['pos']>ss['neg']:max_sentiment=1

    #bill terms
    votes=bill[-1] if bill[-1] else bill[-2]
    relevant_terms={stem.stem(t) for t in cleaner(bill[1])}
    
    #candidate loop
    for id, info in new_info.items():
        #candidate industries:
        relevant_industries=set()
        relevant_industries.update(sum([list(cleaner(i['industry_name'])) for i in info],[]))
        relevant_industries = {stem.stem(t) for t in relevant_industries}
        #relevant?:
        if relevant_terms & relevant_industries:
            #scores move in intervals of 1
            cast=votes.get(id)
            if cast in pos_votes:adjust=1*max_sentiment
            elif cast in neg_votes:adjust=-1*max_sentiment
            for term in relevant_terms & relevant_industries:
                for industry in term_to_industry[term]:
                    if lobbying_scores[id].get(industry):
                        lobbying_scores[id][industry][0]+=max(adjust,0) #-1 -> 0
                        lobbying_scores[id][industry][1]+=1 #total        

In [21]:
lobbying_scores

{'C001054': {'Retired': [2, 2],
  'Real Estate': [0, 0],
  'Leadership PACs': [2, 3],
  'Casinos/Gambling': [0, 0],
  'Misc Defense': [7, 8],
  'Crop Production &amp; Basic Processing': [2, 2],
  'Lawyers/Law Firms': [2, 5],
  'Lobbyists': [0, 0],
  'Building Materials &amp; Equipment': [13, 19],
  'Insurance': [0, 0]},
 'M001212': {'Crop Production &amp; Basic Processing': [2, 2],
  'Retired': [2, 2],
  'Real Estate': [0, 0],
  'Leadership PACs': [2, 3],
  'Republican/Conservative': [1, 1],
  'Building Materials &amp; Equipment': [12, 19],
  'General Contractors': [1, 2],
  'Misc Defense': [6, 8],
  'Health Professionals': [8, 19],
  'Agricultural Services/Products': [24, 32]},
 'R000575': {'Misc Defense': [7, 8],
  'Defense Aerospace': [7, 8],
  'Lobbyists': [0, 0],
  'Leadership PACs': [2, 3],
  'Defense Electronics': [8, 9],
  'Lawyers/Law Firms': [3, 5],
  'Real Estate': [0, 0],
  'Crop Production &amp; Basic Processing': [2, 2],
  'Insurance': [0, 0],
  'General Contractors': [0,

In [22]:
with open('./lobbying_scores.json', 'w') as f:
    json.dump(lobbying_scores,f)